<a href="https://colab.research.google.com/github/jeetpatel1405/NodeApp/blob/main/OpenCVKCF_MIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OpenCV Motion Tracking:  KCF and MIL
This tutorial will help you run opencv KCF and MIL Object tracking on Colab.

MIL Tracking paper:

https://faculty.ucmerced.edu/mhyang/papers/cvpr09a.pdf

KCF paper:

https://openaccess.thecvf.com/content_cvpr_2018/papers/Tang_High-Speed_Tracking_With_CVPR_2018_paper.pdf

This tutorial read a video file and an initial bounding box, apply the opencv implementation of the tracker and display the output video.

In OpenCV, KCF (Kernelized Correlation Filter) and MIL (Multiple Instance Learning) are two types of **tracking** algorithms used for following objects in video sequences. Each has unique characteristics and is suitable for different tracking needs.

###1. KCF (Kernelized Correlation Filter)
**Overview:** KCF is an efficient object-tracking method that uses kernelized correlation filters. It’s known for being fast and effective, especially for real-time applications. KCF is based on the principles of correlation filtering, which allows it to predict object positions in successive frames with high accuracy.

**Key Characteristics:**
*  **Speed:** KCF is generally faster than many other trackers, making it suitable for high-frame-rate applications.
*  **Accuracy:** It’s good at handling scale and rotation changes, though it can struggle with significant occlusions (when the object is partially or fully blocked).
*  **Application:** Works well for relatively stable tracking conditions, such as tracking faces or objects in sports videos.
*  **Limitations:** Struggles in cases where the object is significantly occluded or changes appearance rapidly.

###2. MIL (Multiple Instance Learning)
**Overview:** The MIL tracker uses an approach based on multiple instance learning, where it learns the object’s appearance over time from a set of positive and negative samples around the initial object location. MIL is more robust to partial occlusions and object deformations.

**Key Characteristics:**
*  **Adaptability:** It can handle some degree of occlusion or background clutter, as it updates the object model continuously.
* **Robustness:** MIL is relatively robust in environments with partial occlusions or variable lighting.
* **Application:** Useful for tracking objects with less predictable movements or in situations with moderate occlusions.
* **Limitations:** MIL can be slower than KCF and may suffer from “drift,” where the tracker loses the target if it is occluded for an extended period.


###**Choosing Between KCF and MIL**
*  KCF is preferred for faster, stable tracking when objects remain mostly visible and do not experience major occlusions.
*  MIL is chosen when the object might be partially occluded or if the tracking environment has a lot of background clutter.


Both are accessible in OpenCV’s cv2.Tracker module and can be used with minimal setup in video tracking applications.




## Step 1: Import Dependencies and download video

In [ ]:
import cv2

!wget https://chateaut.fr/documents/Bean_clip.avi

--2024-11-14 02:06:43--  https://chateaut.fr/documents/Bean_clip.avi
Resolving chateaut.fr (chateaut.fr)... 213.186.33.24
Connecting to chateaut.fr (chateaut.fr)|213.186.33.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 561913 (549K) [video/x-msvideo]
Saving to: ‘Bean_clip.avi’

Bean_clip.avi       100%[===================>] 548.74K   474KB/s    in 1.2s    

2024-11-14 02:06:46 (474 KB/s) - ‘Bean_clip.avi’ saved [561913/561913]



## Step 2: init
Two trackers can be tested: KCF and MIL

<font color=red> change the tracker_type to be either MIL or KCF ...currently set to MIL</font>

In [ ]:
# download yolov4 model weights to data folder
# Set up tracker.
tracker_types = ['MIL','KCF']
tracker_type = tracker_types[0] #0 for MIL and 1 for KCF
print('Tracking with: ',tracker_type)

if tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create()  #opencv create MIL Tracker
if tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create()  #openCV create KFC Tracker.




Tracking with:  MIL


In [ ]:
# CODE FOR KCVF
# download yolov4 model weights to data folder
# Set up tracker.
tracker_types = ['MIL','KCF']
tracker_type = tracker_types[1] #0 for MIL and 1 for KCF
print('Tracking with: ',tracker_type)

if tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create()  #opencv create MIL Tracker
if tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create()  #openCV create KFC Tracker.




Tracking with:  KCF


## Step 3: Main loop

In [ ]:
# Init video reader and writer
cap = cv2.VideoCapture('Bean_clip.avi')
#
#Initializing a bounding box
#bbox = img
  ## Bean Tracking
#bbox =  (416, 206, 128, 124) # Children
#bbox =  (379, 33, 89, 106)   # Bean
bbox =  (114, 99, 125, 100)  # Women
#
res=(int(cap.get(3)),int(cap.get(4))) #resolution
fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
!rm output.mp4
out = cv2.VideoWriter('output.mp4', fourcc, 10.0, res)
# Init Tracker
status, frame = cap.read()
ok = tracker.init(frame, bbox)
while(1):
    status, frame = cap.read()
    if status==False:
      break
    timer = cv2.getTickCount()
    ok, bbox = tracker.update(frame)
    #print('tracker status: ',ok)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
    if ok:
      p1 = (int(bbox[0]), int(bbox[1]))
      p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
      cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
      # Tracking failure
      cv2.putText(frame, "Tracking failure detected", (100,80),
      cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
      cv2.putText(frame, tracker_type + " Tracker", (100,20),
      cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

    # Display FPS on frame
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50),
    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

    # Display result
    out.write(frame)
out.release()
cap.release()
print('Traking achieved')

Traking achieved


## Step 4: display output


In [ ]:
# define helper function to display videos
import io
from IPython.display import HTML
from base64 import b64encode
def show_video(file_name, width=640):
  # show resulting deepsort video
  mp4 = open(file_name,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width="{0}" controls>
        <source src="{1}" type="video/mp4">
  </video>
  """.format(width, data_url))

In [ ]:
# convert resulting video from avi to mp4 file format
import os
!rm output2.mp4
#path_video = os.path.join("outputs","tracker.avi")
#%cd outputs/
!ffmpeg -y -loglevel panic -i output.mp4 output2.mp4
#%cd ..

# output object tracking video
#path_output = os.path.join("outputs","output.mp4")
#!ls -al
show_video('output2.mp4', width=640)